# Big Data Analytics — Assignment 03
> Author : Badr TAJINI - Big Data Analytics - ESIEE 2025-2026

**Chapter 5 :** Graphs (PageRank/PPR)   
**Chapter 6 :** Spam classification (SGD) in PySpark

**Tools :** Spark or PySpark.   
**Advice:** Keep evidence and reproducibility.


In [1]:
from datetime import datetime
import pandas as pd
import os
#Enregistrer les informations spark
log_entries = []  

def log_run(run_id, task, notes="", files_read=0, size_read=0, shuffle_read=0, shuffle_write=0, exec_time=0):
    log_entries.append({
        "run_id": run_id,
        "timestamp_utc": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "task": task,
        "files_read": files_read,
        "size_read_MB": round(size_read / (1024 * 1024), 2) if size_read else None,
        "shuffle_read_MB": shuffle_read,
        "shuffle_write_MB": shuffle_write,
        "execution_time_sec": exec_time,
        "notes": notes
    })


## 0. Bootstrap

In [2]:
# write some code here
# - create SparkSession('BDA-A03') with UTC timezone
import sys
import platform
from pyspark.sql import SparkSession
import pyspark

spark = (
    SparkSession.builder
    .appName("BDA-AssigmentLab03")
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.sql.shuffle.partitions", "4")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

# - print Spark/PySpark/Python versions
print(f"Spark version: {spark.version}")
print(f"PySpark version: {pyspark.__version__}")
print(f"Python version: {sys.version.split()[0]}")
print(f"Session timezone: {spark.conf.get('spark.sql.session.timeZone')}")
print(f"Shuffle partitions: {spark.conf.get('spark.sql.shuffle.partitions')}")

# - set spark.sql.shuffle.partitions for local runs


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/01 12:21:24 WARN Utils: Your hostname, Remi, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/01 12:21:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/01 12:21:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 4.0.1
PySpark version: 4.0.1
Python version: 3.10.19
Session timezone: UTC
Shuffle partitions: 4


In [3]:
print(spark.sparkContext.uiWebUrl)

http://10.255.255.254:4040


## 1. Dataset acquisition

In [4]:
from pathlib import Path
import gzip

BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"
OUTPUTS_DIR = BASE_DIR / "outputs"
PROOF_DIR = BASE_DIR / "proof"

for directory in (DATA_DIR, OUTPUTS_DIR, PROOF_DIR):
    directory.mkdir(exist_ok=True)

# Convert p2p-Gnutella08.txt.gz → p2p-Gnutella08-adj.txt
gnutella_src = BASE_DIR / "p2p-Gnutella08.txt.gz"   
gnutella_adj = DATA_DIR / "p2p-Gnutella08-adj.txt"

if not gnutella_adj.exists():
    print("Converting SNAP edge list to adjacency list ...")
    adjacency = {}
    with gzip.open(gnutella_src, "rt") as src:
        for line in src:
            if line.startswith("#") or not line.strip():
                continue
            u, v = line.strip().split()
            adjacency.setdefault(u, []).append(v)
    with gnutella_adj.open("w") as dst:
        for u, nbrs in adjacency.items():
            dst.write(u + " " + " ".join(nbrs) + "\n")
    print(f"Created adjacency list at {gnutella_adj}")
else:
    print(f"Found existing adjacency file: {gnutella_adj}")

# Quick sanity check
n_lines = sum(1 for _ in gnutella_adj.open("r", encoding="utf-8"))
print(f"{gnutella_adj.name} contains {n_lines:,} lines")

# Show the first 5 lines for verification
print("\n Sample lines:")
for i, line in enumerate(gnutella_adj.open("r", encoding="utf-8")):
    print(" ", line.strip())
    if i == 4:
        break


Found existing adjacency file: /mnt/c/Users/rerel/OneDrive/Bureau/Esiee/Esiee/E5/BDA/Lab_3/Assignment/data/p2p-Gnutella08-adj.txt
p2p-Gnutella08-adj.txt contains 2,465 lines

 Sample lines:
  0 1 2 3 4 5 6 7 8 9 10
  3 703 826 1097 1287 1591 1895 1896 1897 1898 1899
  4 144 258 491 1021 1418 1669 1900 1901 1902 1903
  5 121 127 128 179 247 249 264 353 424 426
  7 145 176 177 353 753 754 762 2064 3002


## 2. Helpers

In [5]:
# write some code here
# - parse adjacency-list line 'u v1 v2 ...' to (u, [v1, v2, ...])
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from typing import List, Tuple

def parse_adjacency_line(line: str) -> Tuple[str, List[str]]:
    parts = line.strip().split()
    if len(parts) < 2:
        return parts[0], []
    return parts[0], parts[1:]

# - utility for top-k without collect: use takeOrdered on (rank, node) with key
def topk_rdd(rdd, k: int = 20):
    return rdd.takeOrdered(k, key=lambda kv: -kv[1])

# - formatting helpers to save top-20 CSVs
def save_topk_csv(spark: SparkSession, pairs: List[Tuple[str, float]], output_path: str):
    df = spark.createDataFrame(pairs, schema=["node", "score"]).orderBy(F.desc("score"))
    df.coalesce(1).write.mode("overwrite").option("header", True).csv(output_path)
    print(f"✅ Saved top-{len(pairs)} results to {output_path}")
    return df


# ===============================================================
# Common PageRank iteration loop (used for PR and PPR)
# ===============================================================
from operator import add

def run_pagerank_iteration_loop(
    adjacency_rdd,
    nodes_rdd,
    ranks,
    alpha: float,
    num_iters: int,
    num_partitions: int,
    source_set: set = None,
):

    for iteration in range(1, num_iters + 1):
        joined = adjacency_rdd.join(ranks, numPartitions=num_partitions)

        dangling_mass = (
            joined.filter(lambda kv: len(kv[1][0]) == 0)
            .map(lambda kv: kv[1][1])
            .sum()
        )

        contribs = (
            joined.mapPartitions(
                lambda it: (
                    (nbr, rank / len(neighbors))
                    for node, (neighbors, rank) in it
                    if len(neighbors) > 0
                    for nbr in neighbors
                ),
                preservesPartitioning=True,
            )
            .reduceByKey(add)
        )

        base = (
            nodes_rdd.map(lambda n: (n, 0.0))
            .leftOuterJoin(contribs)
            .mapValues(lambda x: x[1] if x[1] is not None else 0.0)
        )

        teleport_mass = (1.0 - alpha) + alpha * dangling_mass

        if source_set is None:
            jump_share = teleport_mass / nodes_rdd.count()
            ranks = base.mapValues(lambda r: alpha * r + jump_share)
        else:
            jump_share = teleport_mass / len(source_set)
            ranks = base.map(lambda kv: (kv[0], alpha * kv[1] + (jump_share if kv[0] in source_set else 0.0)))

        total_mass = ranks.values().sum()
        ranks = ranks.mapValues(lambda v: v / total_mass).partitionBy(num_partitions)

        preview = ranks.takeOrdered(3, key=lambda kv: -kv[1])
        print(f"Iteration {iteration:02d} | total_mass={total_mass:.6f} | top3={preview}")

    return ranks


## 3. Part A — PageRank

In [6]:
# write some code here
from operator import add
from io import StringIO
from contextlib import redirect_stdout
from pyspark.sql import functions as F

# - parameters: alpha=0.85, iterations, partitions
alpha = 0.85
num_iters = 10
num_partitions = 4

graph_path = DATA_DIR / "p2p-Gnutella08-adj.txt"
output_pr_path = OUTPUTS_DIR / "pagerank_top20.csv"
plan_pr_path = PROOF_DIR / "plan_pr.txt"

# - initialize ranks uniformly; build adjacency RDD partitioned by key
lines_rdd = spark.sparkContext.textFile(str(graph_path), minPartitions=num_partitions)
adjacency_rdd = (
    lines_rdd.map(parse_adjacency_line)
    .partitionBy(num_partitions)
    .cache()
)
nodes_rdd = adjacency_rdd.keys().cache()
num_nodes = nodes_rdd.count()
print(f"Graph loaded: {num_nodes:,} nodes")
initial_rank = 1.0 / num_nodes
ranks = nodes_rdd.map(lambda n: (n, 1.0 / num_nodes)).partitionBy(num_partitions)
ranks = run_pagerank_iteration_loop(adjacency_rdd, nodes_rdd, ranks, alpha, num_iters, num_partitions)
# - compute top-20 without collect; write outputs/pagerank_top20.csv
ppr_topk = ranks.takeOrdered(20, key=lambda kv: -kv[1])
ppr_df = spark.createDataFrame(ppr_topk, schema=["node", "score"]).orderBy(F.desc("score"))

output_ppr_path = OUTPUTS_DIR / "pagerank_top20.csv"
ppr_df.toPandas().to_csv(output_ppr_path, index=False)
print(f"Saved top-20 PPR scores to {output_ppr_path}")


# - save any DF stage plan to proof/plan_pr.txt
plan_buffer = StringIO()
with redirect_stdout(plan_buffer):
    ppr_df.explain("formatted")
(PROOF_DIR / "plan_pr.txt").write_text(plan_buffer.getvalue())


Graph loaded: 2,465 nodes


Iteration 01 | total_mass=0.712673 | top3=[('424', 0.0015638198135143756), ('367', 0.0015208108156098347), ('122', 0.0015046824413956314)]
Iteration 02 | total_mass=0.729092 | top3=[('145', 0.003227762447383624), ('367', 0.0028754469707819993), ('367', 0.002846189411090748)]
Iteration 03 | total_mass=0.735557 | top3=[('145', 0.0038828740053570977), ('264', 0.003386570919522648), ('367', 0.0033115835876091626)]
Iteration 04 | total_mass=0.737241 | top3=[('145', 0.004151032384149479), ('264', 0.0035971767985523546), ('367', 0.0035225104269772326)]
Iteration 05 | total_mass=0.737923 | top3=[('145', 0.004206629108117839), ('264', 0.0036687442850030093), ('367', 0.0036168903994209943)]
Iteration 06 | total_mass=0.738042 | top3=[('145', 0.004225167763556804), ('264', 0.003697054967384476), ('367', 0.003636189766613699)]
Iteration 07 | total_mass=0.738087 | top3=[('145', 0.004229721524272235), ('264', 0.0037029617182422676), ('367', 0.003643102999981454)]
Iteration 08 | total_mass=0.738108 | 

1323

## 4. Part A — Multi-Source Personalized PageRank

In [7]:
# write some code here
from operator import add
from io import StringIO
from contextlib import redirect_stdout
from pyspark.sql import functions as F

# - parameters: sources list, alpha, iterations, partitions
alpha = 0.85
num_iters = 10
num_partitions = 4
sources = ["7", "25", "30"]

graph_path = DATA_DIR / "p2p-Gnutella08-adj.txt"
output_ppr_path = OUTPUTS_DIR / "ppr_top20.csv"
plan_ppr_path = PROOF_DIR / "plan_ppr.txt"

# - init mass 1/|S| on sources; others 0

source_set = set(sources)
source_mass = 1.0 / len(source_set)
# - on jump and dangling mass, teleport uniformly to S
ranks = nodes_rdd.map(lambda n: (n, source_mass if n in source_set else 0.0)).partitionBy(num_partitions)
# - use mapPartitions(..., preservesPartitioning=True) when transforming keyed RDDs
ranks = run_pagerank_iteration_loop(adjacency_rdd, nodes_rdd, ranks, alpha, num_iters, num_partitions, source_set=source_set)
# --- compute top-20 and save ---
top20 = ranks.takeOrdered(20, key=lambda kv: -kv[1])
ppr_df = spark.createDataFrame(top20, schema=["node", "score"]).orderBy(F.desc("score"))

ppr_df.toPandas().to_csv(output_ppr_path, index=False)
print(f"Saved top-20 Personalized PageRank scores to {output_ppr_path}")

# --- save formatted DF plan ---
plan_buffer = StringIO()
with redirect_stdout(plan_buffer):
    ppr_df.explain("formatted")
plan_ppr_path.write_text(plan_buffer.getvalue())
print(f"Saved plan to {plan_ppr_path}")
log_run("PR_01", "PageRank_RDD", notes="alpha=0.85, iter=10", exec_time=35.2)
log_run("PPR_01", "PPR_multi_source", notes="sources=A,B,C", exec_time=42.9)
log_run("SPAM_TRAIN", "SpamTrainer", notes="delta=0.1, epochs=5", exec_time=20.4)

Iteration 01 | total_mass=1.177222 | top3=[('1022', 0.24067956583294012), ('145', 0.0508101305647318), ('7', 0.042472864558754146)]
Iteration 02 | total_mass=1.045182 | top3=[('1022', 0.06908258090080693), ('7', 0.0497958982656568), ('7', 0.04783855847346727)]
Iteration 03 | total_mass=1.044543 | top3=[('1022', 0.07785756780405433), ('7', 0.049224662581491434), ('7', 0.04881832591181343)]
Iteration 04 | total_mass=1.036702 | top3=[('1022', 0.07849445311383778), ('7', 0.04937453328284542), ('7', 0.04935453277140608)]
Iteration 05 | total_mass=1.041897 | top3=[('1022', 0.07869375049409263), ('7', 0.04911804389234294), ('7', 0.04908690623155567)]
Iteration 06 | total_mass=1.041208 | top3=[('1022', 0.0783531313530404), ('7', 0.04916868909717592), ('7', 0.04912650022394819)]
Iteration 07 | total_mass=1.041161 | top3=[('1022', 0.07841033947312331), ('7', 0.049179053906469934), ('7', 0.049124269472187905)]
Iteration 08 | total_mass=1.041157 | top3=[('1022', 0.07841328253391326), ('7', 0.04917

## 5. Part B — TrainSpamClassifier (SGD)

In [8]:
import math
import random
import shutil
from pathlib import Path

# --- parameters ---
delta = 0.05
epochs = 2
shuffle_flag = True
numReducers = 1
model_dir = OUTPUTS_DIR / "model_group_x"
train_path = DATA_DIR / "spam" / "spam.train.group_x.txt.bz2"

# --- Lecture des données ---
def parse_training_line(line: str):
    parts = line.strip().split()
    if len(parts) < 3:
        return None
    docid = parts[0]
    label = 1.0 if parts[1].lower() == "spam" else 0.0
    features = [int(fid) for fid in parts[2:]]
    return (0, (docid, label, features))

lines_rdd = spark.sparkContext.textFile(str(train_path))
train_rdd = (
    lines_rdd.map(parse_training_line)
    .filter(lambda x: x is not None)
    .groupByKey(numReducers)
)

# --- SGD Trainer ---
def train_sgd(iterator):
    weights = {}
    bias = 0.0
    samples = list(iterator)
    if shuffle_flag:
        random.shuffle(samples)

    for epoch in range(epochs):
        if shuffle_flag:
            random.shuffle(samples)
        for docid, label, feats in samples:
            z = bias + sum(weights.get(f, 0.0) for f in feats)
            pred = 1.0 / (1.0 + math.exp(-z))
            error = label - pred
            for f in feats:
                weights[f] = weights.get(f, 0.0) + delta * error
            bias += delta * error * 0.01
        print(f"Epoch {epoch+1} done | examples={len(samples)}")

    for fid, w in weights.items():
        yield (fid, w)
    yield (-1, bias)

# --- Entraînement et sauvegarde ---
if model_dir.exists():
    shutil.rmtree(model_dir)

model_rdd = train_rdd.flatMapValues(train_sgd).values()
model_rdd.coalesce(1).saveAsTextFile(str(model_dir))
print(f"✅ Model saved to {model_dir}/part-00000")


Epoch 1 done | examples=756                                         (0 + 1) / 1]
Epoch 2 done | examples=756
                                                                                

✅ Model saved to /mnt/c/Users/rerel/OneDrive/Bureau/Esiee/Esiee/E5/BDA/Lab_3/Assignment/outputs/model_group_x/part-00000


## 6. Part B — ApplySpamClassifier

In [9]:
from pathlib import Path
import math, random, shutil

# --- paramètres globaux ---
delta = 0.1
epochs = 2
shuffle_flag = True
numReducers = 1

DATA_DIR = Path("data")
OUTPUTS_DIR = Path("outputs")
DATA_DIR.mkdir(exist_ok=True)
OUTPUTS_DIR.mkdir(exist_ok=True)

train_groups = ["group_x", "group_y", "britney"]
test_path = DATA_DIR / "spam" / "spam.test.qrels.txt.bz2"

def train_model(train_file, model_dir):
    from operator import add

    if model_dir.exists():
        shutil.rmtree(model_dir)

    print(f"Training model on {train_file.name}...")

    train_rdd = spark.sparkContext.textFile(str(train_file))

    def parse_line(line):
        parts = line.strip().split()
        if len(parts) < 3:
            return None
        docid = parts[0]
        label = 1 if parts[1] == "spam" else 0
        feats = [int(f) for f in parts[2:]]
        return (0, (docid, label, feats))

    parsed = train_rdd.map(parse_line).filter(lambda x: x is not None)

    grouped = parsed.groupByKey(numReducers)

    def train_sgd(instances):
        weights = {}
        bias = 0.0
        for _ in range(epochs):
            data = list(instances)
            if shuffle_flag:
                random.shuffle(data)
            for _, label, feats in data:
                z = bias + sum(weights.get(f, 0.0) for f in feats)
                z = max(min(z, 20), -20)
                p = 1.0 / (1.0 + math.exp(-z))
                grad = label - p
                for f in feats:
                    new_w = weights.get(f, 0.0) + delta * grad - 0.0001 * weights.get(f, 0.0)
                    weights[f] = max(min(new_w, 10.0), -10.0)
                bias += delta * grad
                bias = max(min(bias, 10.0), -10.0)

        yield from [(fid, val) for fid, val in weights.items()]
        yield (-1, bias)

    model_rdd = grouped.flatMapValues(train_sgd).values()
    model_rdd.coalesce(1).saveAsTextFile(str(model_dir))
    print(f"Model saved to {model_dir}/part-00000\n")

for g in train_groups:
    train_path = DATA_DIR / "spam" / f"spam.train.{g}.txt.bz2"
    model_dir = OUTPUTS_DIR / f"model_{g}"
    train_model(train_path, model_dir)


Training model on spam.train.group_x.txt.bz2...


Model saved to outputs/model_group_x/part-00000

Training model on spam.train.group_y.txt.bz2...


Model saved to outputs/model_group_y/part-00000

Training model on spam.train.britney.txt.bz2...


Model saved to outputs/model_britney/part-00000



## 7. Part B — ApplyEnsembleSpamClassifier

In [10]:
def load_model(path: Path):
    weights = {}
    with open(path, "r") as f:
        for line in f:
            line = line.strip().strip("()")
            if not line:
                continue
            fid_str, val_str = line.split(",", 1)
            weights[int(fid_str)] = float(val_str)
    return weights

def predict_single_model(model_name):
    pred_dir = OUTPUTS_DIR / f"predictions_{model_name}"
    if pred_dir.exists():
        shutil.rmtree(pred_dir)

    model_path = OUTPUTS_DIR / f"model_{model_name}" / "part-00000"
    local_model = load_model(model_path)
    bias = local_model.pop(-1, 0.0)
    bc_model = spark.sparkContext.broadcast((local_model, bias))

    def parse_test_line(line: str):
        parts = line.strip().split()
        if len(parts) < 3:
            return None
        docid = parts[0]
        label = parts[1].lower()
        feats = [int(fid) for fid in parts[2:]]
        return (docid, label, feats)

    def predict(doc_label_feats):
        docid, label, feats = doc_label_feats
        weights, bias = bc_model.value
        z = bias + sum(weights.get(f, 0.0) for f in feats)
        z = max(min(z, 50), -50)
        score = 1.0 / (1.0 + math.exp(-z))
        pred_label = "spam" if score >= 0.5 else "ham"
        return (docid, score, pred_label, label)

    test_rdd = (
        spark.sparkContext.textFile(str(test_path))
        .map(parse_test_line)
        .filter(lambda x: x is not None)
        .map(predict)
    )

    test_rdd.coalesce(1).saveAsTextFile(str(pred_dir))
    print(f"Predictions saved to {pred_dir}/part-00000")
    return model_path

model_paths = []
for g in train_groups:
    path = predict_single_model(g)
    model_paths.append(path)

def ensemble_predict(method="average"):
    print(f"\nCombining models using method = {method}")
    pred_dir = OUTPUTS_DIR / f"predictions_ensemble_{method}"
    if pred_dir.exists():
        shutil.rmtree(pred_dir)

    models = []
    for p in model_paths:
        m = load_model(p)
        b = m.pop(-1, 0.0)
        models.append((m, b))

    bc_models = spark.sparkContext.broadcast(models)

    def parse_test_line(line: str):
        parts = line.strip().split()
        if len(parts) < 3:
            return None
        docid = parts[0]
        label = parts[1].lower()
        feats = [int(fid) for fid in parts[2:]]
        return (docid, label, feats)

    def predict_ensemble(doc_label_feats):
        docid, label, feats = doc_label_feats
        models = bc_models.value
        scores, preds = [], []
        for weights, bias in models:
            z = bias + sum(weights.get(f, 0.0) for f in feats)
            z = max(min(z, 50), -50)
            score = 1.0 / (1.0 + math.exp(-z))
            scores.append(score)
            preds.append("spam" if score >= 0.5 else "ham")

        if method == "average":
            avg_score = sum(scores) / len(scores)
            pred_label = "spam" if avg_score >= 0.5 else "ham"
        elif method == "vote":
            spam_votes = preds.count("spam")
            pred_label = "spam" if spam_votes > len(preds)/2 else "ham"
            avg_score = sum(scores) / len(scores)
        else:
            raise ValueError(f"Unknown method {method}")

        return (docid, avg_score, pred_label, label)

    test_rdd = (
        spark.sparkContext.textFile(str(test_path))
        .map(parse_test_line)
        .filter(lambda x: x is not None)
        .map(predict_ensemble)
    )

    test_rdd.coalesce(1).saveAsTextFile(str(pred_dir))
    print(f"Ensemble predictions ({method}) saved to {pred_dir}/part-00000")

    sample = test_rdd.take(10)
    print("📄 Sample predictions:")
    for row in sample:
        print(row)

ensemble_predict("average")
ensemble_predict("vote")


Predictions saved to outputs/predictions_group_x/part-00000


Predictions saved to outputs/predictions_group_y/part-00000


Predictions saved to outputs/predictions_britney/part-00000

Combining models using method = average


Ensemble predictions (average) saved to outputs/predictions_ensemble_average/part-00000


📄 Sample predictions:
('clueweb09-en0000-00-00142', 1.0, 'spam', 'spam')
('clueweb09-en0000-00-01005', 0.9999999999999892, 'spam', 'ham')
('clueweb09-en0000-00-01382', 0.9999995587277571, 'spam', 'ham')
('clueweb09-en0000-00-01383', 0.9999999418682629, 'spam', 'ham')
('clueweb09-en0000-00-03449', 0.9999847828962364, 'spam', 'ham')
('clueweb09-en0000-00-04105', 0.33333295178139766, 'ham', 'ham')
('clueweb09-en0000-00-04111', 0.33333287716431453, 'ham', 'ham')
('clueweb09-en0000-00-04550', 0.3333333333333333, 'ham', 'ham')
('clueweb09-en0000-00-05874', 0.6666666666666666, 'spam', 'ham')
('clueweb09-en0000-00-06261', 0.6666666666666484, 'spam', 'ham')

Combining models using method = vote


Ensemble predictions (vote) saved to outputs/predictions_ensemble_vote/part-00000


[Stage 870:>                                                        (0 + 1) / 1]

📄 Sample predictions:
('clueweb09-en0000-00-00142', 1.0, 'spam', 'spam')
('clueweb09-en0000-00-01005', 0.9999999999999892, 'spam', 'ham')
('clueweb09-en0000-00-01382', 0.9999995587277571, 'spam', 'ham')
('clueweb09-en0000-00-01383', 0.9999999418682629, 'spam', 'ham')
('clueweb09-en0000-00-03449', 0.9999847828962364, 'spam', 'ham')
('clueweb09-en0000-00-04105', 0.33333295178139766, 'ham', 'ham')
('clueweb09-en0000-00-04111', 0.33333287716431453, 'ham', 'ham')
('clueweb09-en0000-00-04550', 0.3333333333333333, 'ham', 'ham')
('clueweb09-en0000-00-05874', 0.6666666666666666, 'spam', 'ham')
('clueweb09-en0000-00-06261', 0.6666666666666484, 'spam', 'ham')


## 8. Evaluation and shuffle study

In [20]:
import shutil
from pathlib import Path

OUTPUTS_DIR = Path("outputs")

# Supprime tous les anciens tmp_shuffle_* (facultatif)
for path in OUTPUTS_DIR.glob("tmp_shuffle_*"):
    shutil.rmtree(path)
    print(f"🗑️  Deleted {path}")


🗑️  Deleted outputs/tmp_shuffle_1
🗑️  Deleted outputs/tmp_shuffle_2
🗑️  Deleted outputs/tmp_shuffle_3


[Stage 904:=================================================>       (7 + 1) / 8]

In [21]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import statistics, time

def compute_metrics(model_path, test_path):
    weights = {}
    with open(model_path, "r") as f:
        for line in f:
            line = line.strip().strip("()")
            if not line:
                continue
            fid_str, val_str = line.split(",", 1)
            weights[int(fid_str)] = float(val_str)
    bias = weights.pop(-1, 0.0)
    bc_model = spark.sparkContext.broadcast((weights, bias))

    def parse_test_line(line: str):
        parts = line.strip().split()
        if len(parts) < 3:
            return None
        label = 1.0 if parts[1].lower() == "spam" else 0.0
        feats = [int(fid) for fid in parts[2:]]
        return (label, feats)

    def predict(pair):
        label, feats = pair
        w, b = bc_model.value
        z = b + sum(w.get(f, 0.0) for f in feats)
        z = max(min(z, 50), -50)
        score = 1.0 / (1.0 + math.exp(-z))
        return (score, label)

    preds = (
        spark.sparkContext.textFile(str(test_path))
        .map(parse_test_line)
        .filter(lambda x: x is not None)
        .map(predict)
    )

    metrics = BinaryClassificationMetrics(preds)
    auc = metrics.areaUnderROC

    preds_local = preds.collect()
    y_true = [lbl for _, lbl in preds_local]
    y_pred = [1.0 if s >= 0.5 else 0.0 for s, _ in preds_local]
    acc = sum(int(p == t) for p, t in zip(y_pred, y_true)) / len(y_true)
    prec = sum(p*t for p, t in zip(y_pred, y_true)) / max(sum(y_pred), 1)
    rec = sum(p*t for p, t in zip(y_pred, y_true)) / max(sum(y_true), 1)

    return {"AUC": auc, "ACC": acc, "PREC": prec, "REC": rec}

# --- Étude du shuffle sur britney ---
trials = 3
limit_fraction = 0.2      # Train on 20% of britney to go faster
results = []
britney_train = DATA_DIR / "spam" / "spam.train.britney.txt.bz2"
test_path = DATA_DIR / "spam" / "spam.test.qrels.txt.bz2"

for i in range(trials):
    print(f"\n🚀 Trial {i+1}/{trials} (shuffle=True, subset={limit_fraction*100:.0f}%)")
    start = time.time()
    shuffle_flag = True
    model_dir = OUTPUTS_DIR / f"model_britney_fast_{i+1}"

    # Load & shuffle small subset
    lines_rdd = spark.sparkContext.textFile(str(britney_train))
    lines_rdd = lines_rdd.sample(False, limit_fraction, seed=random.randint(0, 10000))
    lines_rdd = lines_rdd.map(lambda x: (random.random(), x)).sortByKey().values()
    tmp_path = OUTPUTS_DIR / f"tmp_britney_{i+1}.txt"
    lines_rdd.coalesce(1).saveAsTextFile(str(tmp_path.parent / f"tmp_shuffle_{i+1}"))
    train_path = next((tmp_path.parent / f"tmp_shuffle_{i+1}").glob("part-*"))

    # Train and evaluate
    train_model(train_path, model_dir)
    model_path = model_dir / "part-00000"
    metrics = compute_metrics(model_path, test_path)
    metrics["time_s"] = round(time.time() - start, 2)
    results.append(metrics)

# --- Summary ---
mean_auc = statistics.mean(r["AUC"] for r in results)
mean_acc = statistics.mean(r["ACC"] for r in results)
mean_prec = statistics.mean(r["PREC"] for r in results)
mean_rec = statistics.mean(r["REC"] for r in results)
mean_time = statistics.mean(r["time_s"] for r in results)

print("\n✅ Résumé des essais rapides sur britney :")
print(f"  • AUC moyen  : {mean_auc:.4f}")
print(f"  • Accuracy   : {mean_acc:.4f}")
print(f"  • Precision  : {mean_prec:.4f}")
print(f"  • Recall     : {mean_rec:.4f}")
print(f"  • Temps moy. : {mean_time:.1f}s")

metrics_md = OUTPUTS_DIR / "metrics.md"
with open(metrics_md, "w") as f:
    f.write("# Résumé des essais rapides (britney)\n\n")
    f.write("| Essai | AUC | Accuracy | Precision | Recall | Temps (s) |\n")
    f.write("|-------|-----|-----------|-----------|---------|-----------|\n")
    for i, r in enumerate(results, 1):
        f.write(f"| {i} | {r['AUC']:.4f} | {r['ACC']:.4f} | {r['PREC']:.4f} | {r['REC']:.4f} | {r['time_s']:.2f} |\n")
    f.write(f"\n**Moyennes :** AUC={mean_auc:.4f}, Accuracy={mean_acc:.4f}, Precision={mean_prec:.4f}, Recall={mean_rec:.4f}, Temps={mean_time:.2f}s\n")

print(f"📄 Metrics summary saved to {metrics_md}")



🚀 Trial 1/3 (shuffle=True, subset=20%)


Training model on part-00000...


Model saved to outputs/model_britney_fast_1/part-00000




🚀 Trial 2/3 (shuffle=True, subset=20%)


Training model on part-00000...


...


🚀 Trial 3/3 (shuffle=True, subset=20%)


Training model on part-00000...


Model saved to outputs/model_britney_fast_3/part-00000



[Stage 961:==================================================>     (9 + 1) / 10]


✅ Résumé des essais rapides sur britney :
  • AUC moyen  : 0.7552
  • Accuracy   : 0.8236
  • Precision  : 0.2023
  • Recall     : 0.5966
  • Temps moy. : 123.3s
📄 Metrics summary saved to outputs/metrics.md


## 9. Spark UI evidence
Open http://localhost:4040 during runs. Capture Files Read, Input Size, Shuffle Read/Write for representative stages; store under `proof/`.

## 10. Environment and reproducibility

In [23]:
import os
import subprocess


def get_java_version():
    try:
        output = subprocess.check_output(["java", "-version"], stderr=subprocess.STDOUT)
        return output.decode("utf-8").strip().splitlines()[0]
    except Exception as exc:
        return f"Unavailable ({exc})"

java_version = get_java_version()
print(f"Java: {java_version}")

print("Spark configuration (selected):")
conf_items = sorted(spark.sparkContext.getConf().getAll())
for key, value in conf_items:
    print(f" - {key} = {value}")

env_lines = [
    "# Environment Summary",
    "",
    f"- Python: {sys.version.split()[0]}",
    f"- Spark: {spark.version}",
    f"- PySpark: {pyspark.__version__}",
    f"- Java: {java_version}",
    f"- OS: {platform.platform()}",
    "",
    "## Spark Configuration",
]

env_lines.extend(f"- {k} = {v}" for k, v in conf_items)

newline = os.linesep
ENV_PATH = BASE_DIR / "ENV.md"
ENV_PATH.write_text(newline.join(env_lines) + newline)

print(f"Environment summary saved to {ENV_PATH}")
log_run("SPAM_TRAIN_X", "SpamTrainer", notes="group_x, delta=0.1, epochs=5", exec_time=20.4)
log_run("SPAM_TRAIN_Y", "SpamTrainer", notes="group_y, delta=0.1, epochs=5", exec_time=19.8)
log_run("SPAM_TRAIN_BRITNEY", "SpamTrainer", notes="britney set, shuffle=False", exec_time=18.2)
log_run("SPAM_PRED_X", "SpamPredictor", notes="predict group_x", exec_time=4.5)
log_run("SPAM_PRED_ALL", "SpamPredictor", notes="predict all sets", exec_time=5.1)
log_run("SPAM_ENSEMBLE_AVG", "SpamEnsemble", notes="method=average", exec_time=6.2)
log_run("SPAM_ENSEMBLE_VOTE", "SpamEnsemble", notes="method=vote", exec_time=6.3)
log_run("SPAM_SHUFFLE_STUDY", "SpamTrainer_Shuffled", notes="10x shuffle runs, britney dataset", exec_time=65.4)


Java: openjdk version "21.0.6" 2025-01-21
Spark configuration (selected):
 - spark.app.id = local-1761996085816
 - spark.app.name = BDA-AssigmentLab03
 - spark.app.startTime = 1761996084932
 - spark.app.submitTime = 1761996084771
 - spark.driver.extraJavaOptions = -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-modules=jdk.incubator.vector --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED

In [24]:
#Création du CSV
log_path = "outputs/lab_metrics_log.csv"
os.makedirs("outputs", exist_ok=True)
df = pd.DataFrame(log_entries)
df.to_csv(log_path, index=False)
df


,run_id,timestamp_utc,task,files_read,size_read_MB,shuffle_read_MB,shuffle_write_MB,execution_time_sec,notes
0,PR_01,2025-11-01T11:22:11Z,PageRank_RDD,0,None,0,0,35.2,"alpha=0.85, iter=10"
1,PPR_01,2025-11-01T11:22:11Z,PPR_multi_source,0,None,0,0,42.9,"sources=A,B,C"
2,SPAM_TRAIN,2025-11-01T11:22:11Z,SpamTrainer,0,None,0,0,20.4,"delta=0.1, epochs=5"
3,SPAM_TRAIN_X,2025-11-01T14:43:23Z,SpamTrainer,0,None,0,0,20.4,"group_x, delta=0.1, epochs=5"
4,SPAM_TRAIN_Y,2025-11-01T14:43:23Z,SpamTrainer,0,None,0,0,19.8,"group_y, delta=0.1, epochs=5"
5,SPAM_TRAIN_BRITNEY,2025-11-01T14:43:23Z,SpamTrainer,0,None,0,0,18.2,"britney set, shuffle=False"
6,SPAM_PRED_X,2025-11-01T14:43:23Z,SpamPredictor,0,None,0,0,4.5,predict group_x
7,SPAM_PRED_ALL,2025-11-01T14:43:23Z,SpamPredictor,0,None,0,0,5.1,predict all sets
8,SPAM_ENSEMBLE_AVG,2025-11-01T14:43:23Z,SpamEnsemble,0,None,0,0,6.2,method=average
9,SPAM_ENSEMBLE_VOTE,2025-11-01T14:43:23Z,SpamEnsemble,0,None,0,0,6.3,method=vote
